# 01 – Intent Classifier (rule‑based)


We keep this deterministic and heavily testable. Start with rules; ML later.


In [5]:
#!csharp
#pragma warning disable 1701, 1702
#nullable enable
#load "SharedDefs.csx"
#r "nuget: Microsoft.Extensions.DependencyInjection,   8.0.0"
#r "nuget: Microsoft.Extensions.Logging.Console,    8.0.0"

using System;
using System.Text.RegularExpressions;
using Microsoft.Extensions.Logging;

public class RuleIntentClassifier : IIntentClassifier
{
    private readonly ILogger<RuleIntentClassifier> _log;
    public RuleIntentClassifier(ILogger<RuleIntentClassifier> log) => _log = log;

    public (Intent intent, double confidence) Classify(string query)
    {
        var q = query.ToLowerInvariant();
        if (Regex.IsMatch(q, @"\b(email|e-mail|contact)\b"))
            return (Intent.GetContactInfo, 0.95);
        if (Regex.IsMatch(q, @"\b(hired?|hire date|start(ed)?|joined)\b") &&
            Regex.IsMatch(q, @"\b(before|after|between|since)\b"))
            return (Intent.FilterByHireDate, 0.85);
        if (Regex.IsMatch(q, @"\b(manager|director|lead|supervisor)\b"))
            return (Intent.FilterByRole, 0.8);
        if (Regex.IsMatch(q, @"\b(engineering|marketing|sales|people|hr|it)\b"))
            return (Intent.SearchByDepartment, 0.7);
        return (Intent.Unknown, 0.3);
    }
}

// quick golden tests
var clf = new RuleIntentClassifier(LoggerFactory.Create(b=>b.AddConsole()).CreateLogger<RuleIntentClassifier>());
var samples = new[] {
    "What are Rick and Summer's emails?",
    "Show managers hired after 2020",
    "Anyone in engineering?",
    "hi"
};
foreach (var s in samples)
{
    var (i,c) = clf.Classify(s);
    Console.WriteLine($"{s} -> {i} ({c})");
}


Installed Packages Microsoft.Extensions.DependencyInjection, 8.0.0 Microsoft.Extensions.Logging.Console, 8.0.0

What are Rick and Summer's emails? -> Unknown (0.3)
Show managers hired after 2020 -> FilterByHireDate (0.85)
Anyone in engineering? -> SearchByDepartment (0.7)
hi -> Unknown (0.3)
